In [23]:
from front.state import (
    ConditionNotMet,
    ForbiddenOverwrite,
    ForbiddenWrite,
    State,
    ValueNotSet,
    mk_binder,
)
import pytest


def test_state():

    st_state = dict()

    st_state['_front'] = dict()  # carve out a place to put front state

    state = State(
        state=st_state['_front'],
        forbidden_writes={'foo'},
        forbidden_overwrites={'apple', 'banana'},
        condition_for_key={'apple': list, 'carrot': lambda x: x > 10},
    )

    assert state != dict()  # it's not an empty dict
    # but you can retrieve the dict of key-value pairs, which will be an empty dict
    assert dict(state) == dict()

    with pytest.raises(ForbiddenWrite):
        state['foo'] = 42

    with pytest.raises(ValueError) as err:
        state['apple'] = 42

    assert (
        str(err.value) == 'The value for the apple key must satisfy condition '
        "IsInstanceOf(class_or_tuple=<class 'list'>)"
    )

    state['apple'] = [4, 2]

    with pytest.raises(ForbiddenOverwrite) as err:
        state['apple'] = [4, 2, 1]

    assert str(err.value) == 'Not allowed to write under this key more than once: apple'

    # but this works since I'm trying to write the same value (just ignores it)
    state['apple'] = [4, 2]

    with pytest.raises(ConditionNotMet) as err:
        state['carrot'] = 10

    assert str(err.value)[:62] == (
        'The value for the carrot key must satisfy condition <function '
    )

    state['carrot'] = 11  # the value is > 10 so okay!
    state['carrot'] = 12  # overwrites allowed

    assert dict(state) == {'apple': [4, 2], 'carrot': 12}

    # We give you a .get since you can always have one when you have a ``__getitem__``:
    assert state.get('carrot') == 12
    assert state.get('key_that_does_not_exist', 'my_default') == 'my_default'

    # And then, we'll forward other MutableMapping operations to the underlying state,
    # so if it can handle them, you'll get those too!
    assert len(state) == 2
    assert list(state) == ['apple', 'carrot']
    assert list(state.items()) == [('apple', [4, 2]), ('carrot', 12)]
    assert 'apple' in state
    assert 'kiwi' not in state


def test_binder_simple():
    """This work is to try to add 'auto registering' of bounded variables"""
    d = dict()
    s = mk_binder(state=d)
    assert d == {}
    assert s.foo is ValueNotSet
    s.foo = 42
    assert s.foo == 42
    assert d == {'foo': 42}
    s.foo = 496
    assert s.foo == 496
    assert d == {'foo': 496}
    s.bar = 8128
    assert d == {'foo': 496, 'bar': 8128}


def test_binder(mk_binder=mk_binder):
    """This work is to try to add 'auto registering' of bounded variables"""

    def _test_fresh_state_with_foobar(binder, state):
        assert binder._state == state
        assert state == {}
        assert binder.foo is ValueNotSet
        binder.foo = 42
        assert binder.foo == 42
        assert state == {'foo': 42}
        binder.foo = 496
        assert binder.foo == 496
        assert state == {'foo': 496}
        binder.bar = 8128
        assert state == {'foo': 496, 'bar': 8128}

    # all defaults: no mk_binder arguments: Get a non-exclusive Binder type
    Binder = mk_binder()
    state = dict()
    binder = Binder(state)
    _test_fresh_state_with_foobar(binder, state)

    # give a state to a binder directly get a non-exclusive binder instance.
    state = dict()
    binder = mk_binder(state)
    _test_fresh_state_with_foobar(binder, state)

    # Specify some identifiers, and be only allowed to use those
    Binder = mk_binder(allowed_ids='foo bar')
    state = dict()
    binder = Binder(state)
    _test_fresh_state_with_foobar(binder, state)

    with pytest.raises(AttributeError) as err:
        binder.not_foo
    assert str(err.value) == (
        'That attribute is not in the self._allowed_ids collection: not_foo'
    )

    with pytest.raises(ForbiddenWrite) as err:
        binder.not_foo = -42
    assert str(err.value) == (
        "Can't write there. The id is not in the self._allowed_ids collection: not_foo"
    )

    Binder = mk_binder(allowed_ids='foo bar')
    state = dict()
    binder = Binder(state)

    assert binder._state == state
    assert state == {}
    assert binder.foo is ValueNotSet
    binder.foo = 42
    assert binder.foo == 42
    assert state == {'foo': 42}
    binder.foo = 496
    assert binder.foo == 496
    assert state == {'foo': 496}
    binder.bar = 8128
    assert state == {'foo': 496, 'bar': 8128}

    # TODO: No test of iter here: Should iter reflect state or inclusion list?
    # Here are a few:

    d = dict(gaga=123)
    b = mk_binder(state=d, allowed_ids='foo bar')
    assert list(b) == ['gaga']  # no foo, no bar (should there be?)
    assert b.foo == ValueNotSet
    # still (trying to access non-existing foo, didn't make a difference (should it?):
    assert list(b) == ['gaga']
    b.foo = 42
    assert list(b) == ['gaga', 'foo']
    assert d == {'gaga': 123, 'foo': 42}

    # but though gaga is in the state, and the iter, it's still not accessible through
    # attributes because not in allwed_ids.
    # TODO: What should the behavior actually be?
    with pytest.raises(AttributeError) as err:
        binder.gaga
    assert str(err.value) == (
        'That attribute is not in the self._allowed_ids collection: gaga'
    )

    with pytest.raises(ForbiddenWrite) as err:
        binder.gaga = 'googoo'
    assert str(err.value) == (
        "Can't write there. The id is not in the self._allowed_ids collection: gaga"
    )

    #
    # # give a state to a binder directly get a non-exclusive binder instance.
    # state = dict(foo=42)
    # binder = mk_binder(state)
    # assert binder.foo == 42
    # assert binder.bar == ValueNotSet
    # binder.foo = 1
    # binder.bar = 2
    # assert state == {'foo': 1, 'bar': 2}
    #
    # # Test iteration behavior
    # d = dict(gaga=123)
    # b = mk_binder(state=d)
    # assert list(b) == ['gaga']  # no foo, no bar (should there be?)
    # assert b.foo == ValueNotSet
    # # still (trying to access non-existing foo, didn't make a difference (should it?):
    # assert list(b) == ['gaga']
    # b.foo = 42
    # assert list(b) == ['gaga', 'foo']
    # assert d == {'gaga': 123, 'foo': 42}
    #
    # # Test iteration behavior when we give an inclusion list

In [24]:
st_state = dict()

st_state['_front'] = dict()  # carve out a place to put front state

state = State(
state=st_state['_front'],
forbidden_writes={'foo'},
forbidden_overwrites={'apple', 'banana'},
condition_for_key={'apple': list, 'carrot': lambda x: x > 10},
)

assert state != dict()  # it's not an empty dict
# but you can retrieve the dict of key-value pairs, which will be an empty dict
assert dict(state) == dict()
with pytest.raises(ForbiddenWrite):
    state['foo'] = 42
with pytest.raises(ValueError) as err:
    state['apple'] = 42

assert (
str(err.value) == 'The value for the apple key must satisfy condition '
"IsInstanceOf(class_or_tuple=<class 'list'>)"
)
state['apple'] = [4, 2]
with pytest.raises(ForbiddenOverwrite) as err:
    state['apple'] = [4, 2, 1]

assert str(err.value) == 'Not allowed to write under this key more than once: apple'




# but this works since I'm trying to write the same value (just ignores it)
state['apple'] = [4, 2]

with pytest.raises(ConditionNotMet) as err:
    state['carrot'] = 10

assert str(err.value)[:62] == (
'The value for the carrot key must satisfy condition <function '
)

state['carrot'] = 11  # the value is > 10 so okay!
state['carrot'] = 12  # overwrites allowed
assert dict(state) == {'apple': [4, 2], 'carrot': 12}




# We give you a .get since you can always have one when you have a ``__getitem__``:
assert state.get('carrot') == 12
assert state.get('key_that_does_not_exist', 'my_default') == 'my_default'




# And then, we'll forward other MutableMapping operations to the underlying state,
# so if it can handle them, you'll get those too!
assert len(state) == 2
assert list(state) == ['apple', 'carrot']
assert list(state.items()) == [('apple', [4, 2]), ('carrot', 12)]
assert 'apple' in state
assert 'kiwi' not in state

In [31]:
"""This work is to try to add 'auto registering' of bounded variables"""
d = dict()
s = mk_binder(state=d)
assert d == {}
assert s.foo is ValueNotSet







In [33]:
s.foo

ValueNotSet

In [ ]:
s.foo = 42
assert s.foo == 42
assert d == {'foo': 42}
s.foo = 496
assert s.foo == 496
assert d == {'foo': 496}
s.bar = 8128
assert d == {'foo': 496, 'bar': 8128}


In [26]:
"""This work is to try to add 'auto registering' of bounded variables"""

def _test_fresh_state_with_foobar(binder, state):
    assert binder._state == state
    assert state == {}
    assert binder.foo is ValueNotSet
    binder.foo = 42
    assert binder.foo == 42
    assert state == {'foo': 42}
    binder.foo = 496
    assert binder.foo == 496
    assert state == {'foo': 496}
    binder.bar = 8128
    assert state == {'foo': 496, 'bar': 8128}

# all defaults: no mk_binder arguments: Get a non-exclusive Binder type
Binder = mk_binder()
state = dict()
binder = Binder(state)
_test_fresh_state_with_foobar(binder, state)

# give a state to a binder directly get a non-exclusive binder instance.
state = dict()
binder = mk_binder(state)
_test_fresh_state_with_foobar(binder, state)

# Specify some identifiers, and be only allowed to use those
Binder = mk_binder(allowed_ids='foo bar')
state = dict()
binder = Binder(state)
_test_fresh_state_with_foobar(binder, state)

with pytest.raises(AttributeError) as err:
    binder.not_foo
assert str(err.value) == (
    'That attribute is not in the self._allowed_ids collection: not_foo'
)

with pytest.raises(ForbiddenWrite) as err:
    binder.not_foo = -42
assert str(err.value) == (
    "Can't write there. The id is not in the self._allowed_ids collection: not_foo"
)

Binder = mk_binder(allowed_ids='foo bar')
state = dict()
binder = Binder(state)

assert binder._state == state
assert state == {}
assert binder.foo is ValueNotSet
binder.foo = 42
assert binder.foo == 42
assert state == {'foo': 42}
binder.foo = 496
assert binder.foo == 496
assert state == {'foo': 496}
binder.bar = 8128
assert state == {'foo': 496, 'bar': 8128}

# TODO: No test of iter here: Should iter reflect state or inclusion list?
# Here are a few:

d = dict(gaga=123)
b = mk_binder(state=d, allowed_ids='foo bar')
assert list(b) == ['gaga']  # no foo, no bar (should there be?)
assert b.foo == ValueNotSet
# still (trying to access non-existing foo, didn't make a difference (should it?):
assert list(b) == ['gaga']
b.foo = 42
assert list(b) == ['gaga', 'foo']
assert d == {'gaga': 123, 'foo': 42}

# but though gaga is in the state, and the iter, it's still not accessible through
# attributes because not in allwed_ids.
# TODO: What should the behavior actually be?
with pytest.raises(AttributeError) as err:
    binder.gaga
assert str(err.value) == (
    'That attribute is not in the self._allowed_ids collection: gaga'
)

with pytest.raises(ForbiddenWrite) as err:
    binder.gaga = 'googoo'
assert str(err.value) == (
    "Can't write there. The id is not in the self._allowed_ids collection: gaga"
)

In [28]:
vars(binder)

{'_state': {'foo': 496, 'bar': 8128},
 '_factory': front.state.BoundVal,
 'foo': 496,
 'bar': 8128}

In [30]:
vars(Binder)

mappingproxy({'__module__': 'front.state',
              '__doc__': 'Specific Binder with exclusive identifiers',
              '_allowed_ids': {'bar', 'foo'},
              'foo': BoundVal('foo'),
              'bar': BoundVal('bar')})

In [21]:
# Valentin's proposal
from dataclasses import dataclass 
from typing import Callable, Optional
from front.state import StateType, BoundVal, Identifiers
@dataclass
class Binder:
    state: StateType
    bound_val_factory: Callable
    bound_val_container: type

    def __getattr__(self, k):
        if k not in self.bound_val_container.__dict__:
            setattr(self.bound_val_container, k, self.bound_val_factory(k))
        return getattr(self.bound_val_container, k)

    def __setattr__(self, k, v):
        # Need this "not _state or _factory", or the __init__ won't be able to set
        # _state and _factory
        if k not in {'_state', '_factory'}:
            self.bound_val_container.__dict__['_state'][k] = v
        self.bound_val_container.__dict__[k] = v  # put it in the __dict__ (so it becomes an attribute)

    def __iter__(self):
            yield from self.bound_val_container._state

def mk_binder(
    *identifiers: Identifiers,
    state: Optional[StateType] = None,
    bound_val_factory=BoundVal,
):
    """

    :param identifiers:
    :param bound_val_factory:
    :return:

    >>> Binder = mk_binder('foo bar')
    >>> d = dict()
    >>> b = Binder(d)

    We ``b.foo`` exists, but is not set.

    >>> b.foo
    ValueNotSet

    So let's set it:

    >>> b.foo = 42
    >>> b.foo
    42

    So ``b.foo`` is now set, but the real point is that this assignment was "registered"
    in the state we give the ``Binder``:

    >>> d
    {'foo': 42}

    Wanna see that again?

    >>> b.foo = "I'm bound"
    >>> b.foo
    "I'm bound"
    >>> d
    {'foo': "I'm bound"}

    And same with ``b.bar``:

    >>> b.bar
    ValueNotSet
    >>> b.bar = "me too"
    >>> b.bar
    'me too'
    >>> d
    {'foo': "I'm bound", 'bar': 'me too'}

    A ``Binder`` will also have some useful mapping methods that are linked to the
    underlying ``state``.

    >>> Binder = mk_binder('the', 'variables', 'I', 'want')
    >>> state = dict()
    >>> b = Binder(state)
    >>> list(b)
    []
    >>> b.want  # I see a want, but no value is set
    ValueNotSet
    >>> list(b)  # list still gives me nothing
    []
    >>> b.want = 42  # but if I set a value for want
    >>> list(b)  # I see want in the list
    ['want']
    >>> 'want' in b  # I can do this too
    True
    >>> 'not_in_there' in b
    False
    >>> 'variables' in b  # 'variables' not "there" because not set
    False


    """
    identifiers = ensure_identifiers(*identifiers)

    @dataclass
    class BoundValContainer:
        pass

    binder_kwargs = dict(
        bound_val_factory=bound_val_factory,
        bound_val_container=BoundValContainer
    )
    if state is not None:
        binder_kwargs['state'] = state
        
    return Binder(**binder_kwargs)

In [36]:
from front.state import _Binder, _DynamicBindsMixin
class DynamicBinder(_Binder, _DynamicBindsMixin):
            """Specific Binder with dynamic on-the-fly identifiers"""

Binder = DynamicBinder

In [39]:
b = Binder(state={})

In [40]:
vars(b)

{'_state': {}, '_factory': front.state.BoundVal}

In [41]:
b.foo

ValueNotSet

In [ ]:
@dataclass
class Binder:
    state: StateType
    

    def __getattr__(self, k):
        validate(k)
        return retrieve_from_state(self.state, k)

    def __setattr__(self, k, v):
        # Need this "not _state or _factory", or the __init__ won't be able to set
        # _state and _factory
        validate_kv(k,v)
        set_state(self.state, k)

    #def __iter__(self):
    #        yield from self.bound_val_container._state

In [ ]:
# b.foo means retrieve(state(b),foo)

In [ ]:
# b is an object that (contains) points to a State object state(b).
# an attribute access like b.foo actually means retrieve(state(b),foo)
# foo, bar are descriptors (why? is it mostly to be able to do validation on them), and one should be able to create them on the fly. 
# there is a validation mechanism too 